# RAW cars dataset (uncleaned)

- This repository contains 225 RAW images of scenes containing cars.

## RAW_converted (the folder that contains the uncleaned images we're using)

- The original RAW images have been scaled to [0, 255] from their original scale of [0, 1023] and converted to the lossless PNG image format. This reduces their size and make circulation feasible.

- ### For the sake of simplicity, I'm only reading 5 images for processing

- ### We can see that the 5 images I selected have different shapes

## This is just a simple program to resize an image to (150, 150), the final output images may be stretched or compressed

In [ ]:
import requests
from PIL import Image
import numpy as np
import io
import time

# --- Configuration ---
# Base URL for the raw image files (RAW_converted directory)
BASE_RAW_URL = "https://raw.githubusercontent.com/EkdeepSLubana/raw_dataset/master/RAW_converted/"
# Define the range of images: 1.png to 5.png
IMAGE_IDS = range(1, 6) 

# The uniform size we want for the "Cleaned" dataset
TARGET_SIZE = (150, 150) 
cleaned_dataset = []
successful_loads = 0

print(f"Starting download and resize of {len(IMAGE_IDS)} images (.png format)...")

# --- Iterate through URLs, Load, and Resize ---
for img_id in IMAGE_IDS:
    file_name = f"{img_id}.png"
    full_url = BASE_RAW_URL + file_name
    
    # --- OUTPUT FOR ALL 5 IMAGES ---
    print(f"\nProcessing {file_name}...")
    
    try:
        response = requests.get(full_url, stream=True, timeout=10)
        response.raise_for_status()

        image_bytes = io.BytesIO(response.content)

        # Convert to 'RGB' to ensure 3 channels
        img = Image.open(image_bytes).convert('RGB') 
        
        # --- OUTPUT FOR ALL 5 IMAGES ---
        print(f"  Original size (Uncleaned): {img.size}") 
        
        resized_img = img.resize(TARGET_SIZE)

        image_array = np.array(resized_img)
        cleaned_dataset.append(image_array)
        successful_loads += 1
        
        # --- OUTPUT FOR ALL 5 IMAGES ---
        print(f"  Resized size (Cleaned): {resized_img.size}")

    except requests.exceptions.RequestException as e:
        print(f"Skipping {file_name}: Download error: {e}")
    except IOError:
        print(f"Skipping {file_name}: Failed to open or process image.")
    except Exception as e:
        print(f"Skipping {file_name}: An unexpected error occurred: {e}")

# ... (Final Summary) ...
end_time = time.time()
if cleaned_dataset:
    final_data = np.stack(cleaned_dataset)
    print("\n--- Final Dataset Summary ---")
    print(f"Successfully loaded and resized {successful_loads} images in {end_time - start_time:.2f} seconds.")
    print(f"Final Cleaned Dataset Shape (N, H, W, C): {final_data.shape}")
else:
    print("\n--- Final Dataset Summary ---")
    print("No images were successfully loaded due to network issues. The final shape explanation assumes successful execution.")

- N - This is the batch size, representing the total count of images that were successfully loaded and stacked (1.png through 10.png).
- H - The fixed vertical dimension (height) of every single image, as enforced by the TARGET_SIZE = (150, 150) parameter.
- W - The fixed horizontal dimension (width) of every single image, also enforced by the TARGET_SIZE parameter.
- C - The number of color channels. PNG images typically store color data as RGB (Red, Green, Blue), which requires 3 channels.

### All (selected) images have been resized to (150, 150)

# POSSIBLE ERRORS

## Incorrect File Extension (Should be .png, but requesting .jpg)

In [14]:
url_error_a = BASE_RAW_URL + "1.jpg"
response_a = requests.get(url_error_a)
response_a.raise_for_status() # This will raise an HTTPError (404 Not Found)
print("File A loaded successfully.")

HTTPError: 404 Client Error: Not Found for url: https://raw.githubusercontent.com/EkdeepSLubana/raw_dataset/master/RAW_converted/1.jpg

## Attempting to read a non-image file (e.g., README.md)

In [15]:
# NOTE: Need to adjust BASE_RAW_URL to access the root repo directory
base_url_for_readme = "https://raw.githubusercontent.com/EkdeepSLubana/raw_dataset/master/"
url_error_c = base_url_for_readme + "README.md"

response_c = requests.get(url_error_c)
file_bytes_c = io.BytesIO(response_c.content)

# This will raise a PIL.UnidentifiedImageError or similar IOError
img_c = Image.open(file_bytes_c) 
print("Image C opened successfully.")

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x0000025D1C25B060>

## Invalid Resize Target (Needs tuple, giving an int)
- Here we try using 150 for shape instead of (150, 150)

In [16]:
# Assuming 'img' is a successfully loaded image object
if 'img' in locals():
    try:
        # This will raise a TypeError because 150 is not a tuple
        resized_img_d = img.resize(150) 
        print("Resize D successful.")
    except TypeError as e:
        print(f"Error D: {e}")

Error D: argument 1 must be 2-item sequence, not int


## Attempting to Stack Unresized Images

In [17]:
# Load two images (1.png and 5.png) of different original sizes:
img_1_url = BASE_RAW_URL + "1.png"
img_5_url = BASE_RAW_URL + "5.png"

img_1 = Image.open(io.BytesIO(requests.get(img_1_url).content)).convert('RGB')
img_5 = Image.open(io.BytesIO(requests.get(img_5_url).content)).convert('RGB')

uncleaned_list = [np.array(img_1), np.array(img_5)]

# This will raise a ValueError because the arrays (images) have different shapes
np.stack(uncleaned_list) 
print("Stack E successful.") # This line will not be reached

ValueError: all input arrays must have the same shape

# Old code

### messy-vs-clean-room dataset (uncleaned)

this code doesn't read any actual images, so nevermind

In [19]:
import requests
import numpy as np
import io

github_url = "https://github.com/GuanqiaoDing/messy-room-classifier/blob/master/data/room_dataset.npy"

# Convert the GitHub web URL to the raw file URL
raw_url = github_url.replace("blob", "raw")

# Use the requests library to fetch the content from the raw URL
response = requests.get(raw_url)

# Check if the request was successful
if response.status_code == 200:
    file_bytes = io.BytesIO(response.content)
    
    # Load the NumPy array from the bytes data
    data = np.load(file_bytes, allow_pickle=True)
    
    print("Dataset loaded successfully!")
    print(f"Data shape: {data.shape}")
    print(f"Data type: {type(data)}")
    print(f"First few entries: {data[:2]}")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")


Dataset loaded successfully!
Data shape: (2, 2)
Data type: <class 'numpy.ndarray'>
First few entries: [[array([[[[163, 182, 203],
           [163, 182, 203],
           [162, 181, 202],
           ...,
           [205, 210, 219],
           [205, 210, 219],
           [206, 211, 220]],

          [[163, 182, 203],
           [163, 182, 203],
           [162, 181, 202],
           ...,
           [206, 211, 220],
           [206, 211, 220],
           [208, 213, 222]],

          [[162, 181, 203],
           [163, 182, 203],
           [162, 181, 202],
           ...,
           [206, 211, 220],
           [206, 211, 220],
           [207, 212, 221]],

          ...,

          [[ 31,  32,  28],
           [ 32,  33,  29],
           [ 41,  42,  38],
           ...,
           [ 32,  37,  35],
           [ 32,  37,  36],
           [ 32,  37,  36]],

          [[ 30,  31,  27],
           [ 31,  32,  27],
           [ 35,  39,  33],
           ...,
           [ 32,  36,  34],
          

### importing the MNIST dataset (cleaned)

In [2]:
import numpy as np
import urllib.request
import os
import pandas as pd

url="https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz"
local_path = "mnist.npz"

try:
    # 1. Download the file from the URL
    urllib.request.urlretrieve(url, local_path)
    print("Download complete.")

    # 2. Load the data using numpy
    data = np.load(local_path, allow_pickle=True)

    # 3. Extract the data arrays from the loaded object
    xtrain = data['x_train']
    ytrain = data['y_train']
    xtest = data['x_test']
    ytest = data['y_test']
    
    print("Dataset successfully loaded.")

    # 4. Display shapes to verify
    print(f"x train shape: {xtrain.shape}")
    print(f"y train shape: {ytrain.shape}")
    print(f"x test shape: {xtest.shape}")
    print(f"y test labels shape: {ytest.shape}")

except Exception as e:
    print(f"An error occurred: {e}")

Download complete.
Dataset successfully loaded.
x train shape: (60000, 28, 28)
y train shape: (60000,)
x test shape: (10000, 28, 28)
y test labels shape: (10000,)
